In [1]:
from einops import rearrange
import matplotlib.pyplot as plt
import numpy as np
import torch
from pytorch_memlab import MemReporter

from linop import SubspaceLinopFactory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%load_ext autoreload
%autoreload 2

## Data

In [2]:
# Load MR data
ksp = torch.from_numpy(np.load('data/ksp.npy')).to(device) # Raw k-space data
trj = torch.from_numpy(np.load('data/trj.npy')).to(device) # Trajectory
dcf = torch.from_numpy(np.load('data/dcf.npy')).type(torch.float32).to(device) # Density compensation
phi = torch.from_numpy(np.load('data/phi.npy')).type(torch.complex64).to(device) # Subspace basis
mps = torch.from_numpy(np.load('data/mps.npy')).type(torch.complex64).to(device) # Sensitivity maps

# Fix dimensions
ksp = rearrange(ksp, 'c k 1 t -> t c k')
trj = rearrange(trj, 'k 1 r d -> r d k') * 2 * np.pi
dcf = rearrange(dcf, 'k 1 r -> r k')


# Show dimensions and types
print(f'ksp shape = {ksp.shape}, dtype = {ksp.dtype}')
print(f'trj shape = {trj.shape}, dtype = {trj.dtype}')
print(f'dcf shape = {dcf.shape}, dtype = {dcf.dtype}')
print(f'phi shape = {phi.shape}, dtype = {phi.dtype}')
print(f'mps shape = {mps.shape}, dtype = {mps.dtype}')

ksp shape = torch.Size([600, 16, 1000]), dtype = torch.complex64
trj shape = torch.Size([600, 2, 1000]), dtype = torch.float64
dcf shape = torch.Size([600, 1000]), dtype = torch.float32
phi shape = torch.Size([4, 600]), dtype = torch.complex64
mps shape = torch.Size([16, 200, 200]), dtype = torch.complex64


## Create linops

In [6]:
linop_factory = SubspaceLinopFactory(trj, phi, mps, torch.sqrt(dcf))
linop_factory.to(device)
A, ishape, oshape = linop_factory.get_forward()
AH, _, _ = linop_factory.get_adjoint()
AHA, _, _ = linop_factory.get_normal(toeplitz=True, device=device, verbose=True)

> Computing weights...
>> Time: 0.0010953517630696297 s
> Generating kernels...
>> Calculating kernel(0, 0)
>> Calculating kernel(1, 0)
>> Calculating kernel(2, 0)
>> Calculating kernel(3, 0)
>> Calculating kernel(1, 1)
>> Calculating kernel(2, 1)
>> Calculating kernel(3, 1)
>> Calculating kernel(2, 2)
>> Calculating kernel(3, 2)
>> Calculating kernel(3, 3)
>> Time: 97.5351887345314 s


TypeError: object of type 'NoneType' has no len()

In [4]:
reporter = MemReporter(linop_factory)
reporter.report()

Element type                                            Size  Used MEM
-------------------------------------------------------------------------------
Storage on cuda:0
Tensor0                                   (16, 1000, 1, 600)    73.24M
Tensor1                                       (1000, 1, 600)     2.29M
Tensor2                                             (4, 600)    19.00K
Tensor3                                       (16, 200, 200)     4.88M
Tensor4                                      (600, 16, 1000)     0.00B
Tensor5                                       (600, 2, 1000)     9.16M
Tensor6                                          (600, 1000)     0.00B
trj                                           (600, 2, 1000)     0.00B
phi                                                 (4, 600)     0.00B
mps                                           (16, 200, 200)     0.00B
sqrt_dcf                                         (600, 1000)     2.29M
subsamp_idx                                       

/home/mark/mambaforge/envs/mrf-subspace/lib/python3.10/site-packages/torch/distributed/distributed_c10d.py:283: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
/home/mark/mambaforge/envs/mrf-subspace/lib/python3.10/site-packages/pytorch_memlab/mem_reporter.py:95: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  fact_numel = tensor.storage().size()
/home/mark/mambaforge/envs/mrf-subspace/lib/python3.10/site-packages/pytorch_memlab/mem_reporter.py:104: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tens